# Many simulations at different carrying capacities and switching speeds

With a slight update to the way success rates are calculated.

In [1]:
import pandas as pd
import numpy as np
import math
import os
from collections import OrderedDict
from IPython.display import display, clear_output, Markdown, HTML

from plotly import tools
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
import plotly.graph_objs as go
import plotly.io as pio

from simulator import *
from simulator_plotting import *

init_notebook_mode(connected=True)

In [2]:
pairs = [('CTX', 'SAM'), ('AM', 'AMC'), ('ZOX', 'CXM')]
pairs = [[dataset2.loc[item] for item in pair] for pair in pairs]

In [3]:
def hist_carrying_cap(landscape, k_values=[9, 8, 7, 6], num=100, **kargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    data = []
    for k in k_values:
        param = dict(kargs)
        param['carrying_cap'] = 10**k
        param['prob_mutation'] = 10**(-(k-1))
        times = [simulate(landscape, **param)['T_f'] for i in range(num)]
        times = [t for t in times if t != -1]
        trace = go.Histogram(
            x = times,
            #xbins = dict(
            #    start = 0,
            #    end = 1000,
            #    size = 10),
            #autobinx = False,
            opacity=0.75,
            name = 'K=10^{}'.format(k)
        )
        data.append(trace)
    layout = go.Layout(
        barmode = 'overlay',
        title = 'Running {} simulations on {} at different carrying capacities'.format(num, name),
        xaxis = dict(
            title = 'Time to fixation',
            rangemode='tozero'
            #range = [0, 1000]
        ),
        yaxis = dict(
            title = 'Simulation count'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

## Using histograms to display effect of carrying capacity

In [4]:
hist_carrying_cap(dataset2.loc['CPD'], [9, 7, 5, 3])
hist_carrying_cap(pairs[0], [9, 7, 5, 3], frequency=100)

In [5]:
def hist_switching(landscapes, frequencies=[200, 100, 50], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    landscapes = [ls.tolist() for ls in landscapes]
    data = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        times = [simulate(landscapes, **param)['T_f'] for i in range(num)]
        #print(times)
        times = [t for t in times if t != -1]
        trace = go.Histogram(
            x = times,
            xbins = dict(
                size = 1),
            autobinx = False,
            opacity=0.75,
            name = 'f={}'.format(f)
        )
        data.append(trace)
    layout = go.Layout(
        barmode = 'overlay',
        title = 'Running {} simulations on {} at different switching speeds'.format(num, name),
        xaxis = dict(
            title = 'Time to fixation',
            #rangemode='tozero'
            #range = [0, 1000]
        ),
        yaxis = dict(
            title = 'Simulation count'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

## High switching speeds show large variation in T<sub>f</sub>

Histograms may be less useful in this case. See the box plots below.

In [6]:
hist_switching(pairs[0])
hist_switching(pairs[1])
hist_switching(pairs[2])

In [7]:
def box_switching(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    landscapes = [ls.tolist() for ls in landscapes]
    data = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        times = [simulate(landscapes, **param)['T_f'] for i in range(num)]
        #print(times)
        times = [t for t in times if t != -1]
        trace = go.Box(
            y = times,
            boxpoints = False,
            name = 'f={}'.format(f)
        )
        data.append(trace)
    layout = go.Layout(
        title = 'Running {} simulations on {} at different switching speeds'.format(num, name),
        xaxis = dict(
            title = 'Switching speed',
        ),
        yaxis = dict(
            title = 'Time to fixation'
        )
    )
    fig = go.Figure(data=data, layout=layout)
    iplot(fig, show_link=False)

In [8]:
def highlight_yes(v):
    if v == 'Yes':
        return 'font-weight: bold'
    else:
        return ''
def highlight_max(s):
    is_max = s == s.max()
    return ['font-style: italic' if v else '' for v in is_max]

def pathway_analysis(landscape, num=100, pretty=False, **kwargs):
    if isinstance(landscape, list):
        name = '{} + {}'.format(landscape[0].name, landscape[1].name)
        landscape = [ls.tolist() for ls in landscape]
    else:
        name = landscape.name
        landscape = landscape.tolist()
    #success_count = 0
    greedy_path = ''
    paths = {}
    T_f_sum = 0
    global_optimum = ''
    local_optima = []
    for i in range(num):
        results = simulate(landscape, **kwargs)
        if not global_optimum:
            global_optimum = results['global_optimum']
            local_optima = ', '.join(results['local_optima'])
        if not greedy_path:
            greedy_path = ','.join(results['greedy_path'])
            paths[greedy_path] = [0, 0]
        actual_path = ','.join(results['actual_path'])
        if actual_path in paths:
            paths[actual_path][0] += 1
        else:
            paths[actual_path] = [1, 0] # (appearances, successful appearances)
        if results['T_f'] != -1:
            paths[actual_path][1] += 1
            T_f_sum += results['T_f']
    data = []
    total_success_count = sum([sc for c, sc in paths.values()])
    num_paths = sum([c > 0 for c, sc in paths.values()])
    num_successful_paths = sum([sc > 0 for c, sc in paths.values()])
    for path, (count, success_count) in paths.items():
        successful = success_count > 0
        row = OrderedDict()
        row['Pathway'] = path
        row['Successful?'] = 'Yes' if successful else 'No'
        row['Greedy?'] = 'Yes' if path is greedy_path else 'No'
        row['Number of appearances'] = count
        row['Weight'] = success_count / total_success_count if successful else None
        data.append(row)
    df = pd.DataFrame(data)
    s = df.style.applymap(highlight_yes, subset='Successful?').apply(highlight_max, subset='Weight')
    max_weight = df['Weight'].max()
    if pretty:
        display(s)
        display(Markdown(f'Global optimum: {global_optimum}'))
        display(Markdown(f'Weight of the dominant successful pathway: {max_weight}'))
        display(Markdown(f'Success rate: {num_successful_paths / num_paths}'))
    else:
        return {
            'paths': df,
            'global_optimum': global_optimum,
            'max_weight': max_weight,
            'success_count': total_success_count,
            'success_rate': num_successful_paths / num_paths
        }

In [9]:
def table_switching(landscapes, frequencies=[200,175,150,125,100,75,50,25,20,15,10,5,1], num=100, **kargs):
    name = '{} + {}'.format(landscapes[0].name, landscapes[1].name)
    success_counts = []
    success_rates = []
    weights = []
    for f in frequencies:
        param = dict(kargs)
        param['frequency'] = f
        analysis = pathway_analysis(landscapes, **param)
        success_counts.append(analysis['success_count'])
        success_rates.append(analysis['success_rate'])
        weights.append(analysis['max_weight'])
    df = pd.DataFrame({
        'f': frequencies,
        'Success count': success_counts,
        'Success rate': success_rates,
        'Weight of dominant path': weights
    }).set_index('f')
    return df

## CTX + SAM

### K=10^9

In [10]:
display(table_switching(pairs[0]))
box_switching(pairs[0])

,Success count,Success rate,Weight of dominant path
f,,,
200,100,1.0,1.00
175,100,1.0,1.00
150,100,1.0,1.00
125,100,1.0,1.00
100,100,1.0,1.00
75,100,1.0,0.99
50,100,1.0,0.85
25,81,0.5,1.00
20,10,0.5,1.00


### K=10^6

In [12]:
display(table_switching(pairs[0], carrying_cap=10**6, prob_mutation=10**-5))
box_switching(pairs[0], carrying_cap=10**6, prob_mutation=10**-5)

,Success count,Success rate,Weight of dominant path
f,,,
200,100,1.00,0.820000
175,100,1.00,0.870000
150,100,1.00,0.890000
125,100,1.00,0.910000
100,100,1.00,0.950000
75,100,1.00,0.960000
50,100,1.00,0.940000
25,100,1.00,0.510000
20,100,1.00,0.900000


### K=10^4

In [13]:
display(table_switching(pairs[0], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[0], carrying_cap=10**4, prob_mutation=10**-3)

,Success count,Success rate,Weight of dominant path
f,,,
200,100,1.000000,0.500000
175,100,1.000000,0.520000
150,100,1.000000,0.590000
125,100,1.000000,0.560000
100,100,1.000000,0.450000
75,100,1.000000,0.490000
50,100,1.000000,0.440000
25,100,1.000000,0.520000
20,100,1.000000,0.520000


## AM + AMC

### K=10^9

In [14]:
display(table_switching(pairs[1]))
box_switching(pairs[1])

,Success count,Success rate,Weight of dominant path
f,,,
200,100,1.000000,0.860000
175,100,1.000000,0.920000
150,100,1.000000,0.940000
125,100,1.000000,0.940000
100,100,1.000000,0.840000
75,100,1.000000,0.980000
50,99,0.666667,0.989899
25,10,0.666667,0.700000
20,4,0.500000,0.750000


### K=10^6

In [15]:
display(table_switching(pairs[1], carrying_cap=10**6, prob_mutation=10**-5))
box_switching(pairs[1], carrying_cap=10**6, prob_mutation=10**-5)

,Success count,Success rate,Weight of dominant path
f,,,
200,100,1.0,0.56
175,100,1.0,0.52
150,100,1.0,0.62
125,100,1.0,0.64
100,100,1.0,0.67
75,100,1.0,0.58
50,100,1.0,0.93
25,100,1.0,0.90
20,100,1.0,0.95


### K=10^4

In [16]:
display(table_switching(pairs[1], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[1], carrying_cap=10**4, prob_mutation=10**-3)

,Success count,Success rate,Weight of dominant path
f,,,
200,100,1.0,0.20
175,100,1.0,0.30
150,100,1.0,0.24
125,100,1.0,0.24
100,100,1.0,0.24
75,100,1.0,0.26
50,100,1.0,0.26
25,100,1.0,0.49
20,100,1.0,0.52


## ZOX + CXM

### K=10^9

In [17]:
display(table_switching(pairs[2]))
box_switching(pairs[2])

,Success count,Success rate,Weight of dominant path
f,,,
200,100,1.0,1.00
175,100,1.0,1.00
150,100,1.0,1.00
125,100,1.0,1.00
100,100,1.0,1.00
75,100,1.0,1.00
50,100,1.0,1.00
25,100,1.0,1.00
20,100,1.0,1.00


### K=10^6

In [18]:
display(table_switching(pairs[2], carrying_cap=10**6, prob_mutation=10**-5))
box_switching(pairs[2], carrying_cap=10**6, prob_mutation=10**-5)

,Success count,Success rate,Weight of dominant path
f,,,
200,100,1.0,0.98
175,100,1.0,0.99
150,100,1.0,1.00
125,100,1.0,1.00
100,100,1.0,0.99
75,100,1.0,0.97
50,100,1.0,1.00
25,100,1.0,0.92
20,100,1.0,0.73


### K=10^4

In [19]:
display(table_switching(pairs[2], carrying_cap=10**4, prob_mutation=10**-3))
box_switching(pairs[2], carrying_cap=10**4, prob_mutation=10**-3)

,Success count,Success rate,Weight of dominant path
f,,,
200,100,1.0,0.78
175,100,1.0,0.84
150,100,1.0,0.88
125,100,1.0,0.87
100,100,1.0,0.76
75,100,1.0,0.88
50,100,1.0,0.85
25,100,1.0,0.85
20,100,1.0,0.83


## Carrying capacities below 10^4 do not succeed

The tables below are for K=10^3:

In [20]:
for pair in pairs:
    display(Markdown('__{} + {}__'.format(pair[0].name, pair[1].name)))
    display(table_switching(pair, carrying_cap=10**3, prob_mutation=10**-2))

__CTX + SAM__

,Success count,Success rate,Weight of dominant path
f,,,
200,0,0.000000,NaN
175,0,0.000000,NaN
150,0,0.000000,NaN
125,0,0.000000,NaN
100,0,0.000000,NaN
75,0,0.000000,NaN
50,0,0.000000,NaN
25,0,0.000000,NaN
20,0,0.000000,NaN


__AM + AMC__

,Success count,Success rate,Weight of dominant path
f,,,
200,0,0.0,NaN
175,0,0.0,NaN
150,0,0.0,NaN
125,0,0.0,NaN
100,0,0.0,NaN
75,0,0.0,NaN
50,0,0.0,NaN
25,0,0.0,NaN
20,0,0.0,NaN


__ZOX + CXM__

,Success count,Success rate,Weight of dominant path
f,,,
200,0,0.0,NaN
175,0,0.0,NaN
150,0,0.0,NaN
125,0,0.0,NaN
100,0,0.0,NaN
75,0,0.0,NaN
50,0,0.0,NaN
25,0,0.0,NaN
20,0,0.0,NaN


## Is switching at a frequency of 1 the same as averaging 2 landscapes?

In [21]:
def avg_landscape(pair):
    return [(i + j) / 2 for i, j in zip(pair[0], pair[1])]
display(Markdown('### Average of ZOX and CXM'))
plot_simulation(simulate(avg_landscape(pairs[2])))
pathway_analysis(pd.Series(avg_landscape(pairs[2]), name='avg'), pretty=True)
display(Markdown('### Switching between ZOX and CXM at frequency 1'))
plot_simulation(simulate([l.tolist() for l in pairs[2]], frequency=1))
pathway_analysis(pairs[2], pretty=True, frequency=1)

### Average of ZOX and CXM

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0100,0110,0111",Yes,Yes,98,0.98
1,"0000,0010,0110,0111",Yes,No,2,0.02


Global optimum: 0111

Weight of the dominant successful pathway: 0.98

Success rate: 1.0

### Switching between ZOX and CXM at frequency 1

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0010,0011,0111",No,Yes,0,nan
1,"0000,0100,0110,0111",Yes,No,97,0.97
2,"0000,0010,0110,0111",Yes,No,3,0.03


Global optimum: 0111

Weight of the dominant successful pathway: 0.97

Success rate: 1.0

They are pretty similar!

In [22]:
display(Markdown('### Average of AM and AMC'))
plot_simulation(simulate(avg_landscape(pairs[1]), carrying_cap=10**6, prob_mutation=10**-5))
pathway_analysis(pd.Series(avg_landscape(pairs[1]), name='avg'), pretty=True, carrying_cap=10**6, prob_mutation=10**-5)
display(Markdown('### Switching between AM and AMC at frequency 1'))
plot_simulation(simulate([l.tolist() for l in pairs[1]], frequency=1, carrying_cap=10**6, prob_mutation=10**-5))
pathway_analysis(pairs[1], pretty=True, frequency=1, carrying_cap=10**6, prob_mutation=10**-5)

### Average of AM and AMC

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0001,1001,1101",Yes,Yes,94,0.94
1,"0000,1000,1001,1101",Yes,No,2,0.02
2,"0000,0001,1001,1011,1111,1101",Yes,No,3,0.03
3,"0000,0001,0101,1101",Yes,No,1,0.01


Global optimum: 1101

Weight of the dominant successful pathway: 0.94

Success rate: 1.0

### Switching between AM and AMC at frequency 1

,Pathway,Successful?,Greedy?,Number of appearances,Weight
0,"0000,0010",No,Yes,0,nan
1,"0000,0001,1001,1101",Yes,No,90,0.9
2,"0000,0001,1001,1011,1111,1101",Yes,No,2,0.02
3,"0000,0001,0101,1101",Yes,No,5,0.05
4,"0000,1000,1001,1101",Yes,No,3,0.03


Global optimum: 1101

Weight of the dominant successful pathway: 0.9

Success rate: 1.0